# Lesson 02: Grayscaling

Converting a color image to grayscale reduces it from 3 channels to 1 channel.  
This is one of the most common preprocessing steps in image processing and computer vision.

## Why grayscale?
- Simpler to process (1/3 the data)
- Many algorithms (edge detection, thresholding) work on intensity only
- Removes color information that might not be relevant to the task

## Formula (ITU-R BT.601 luminance):
$$\text{Y} = 0.299 \cdot R + 0.587 \cdot G + 0.114 \cdot B$$

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img_bgr = cv2.imread('sealion_hero.png')
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

## 1. Converting to Grayscale with OpenCV

In [ ]:
# Method 1: cv2.cvtColor (recommended)
gray_cv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

# Method 2: imread with GRAYSCALE flag
gray_load = cv2.imread('sealion_hero.png', cv2.IMREAD_GRAYSCALE)

print(f'Original shape:  {img_bgr.shape}')
print(f'Grayscale shape: {gray_cv.shape}')  # no channel dimension
print(f'Same result?     {np.array_equal(gray_cv, gray_load)}')

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].imshow(img_rgb);               axes[0].set_title('Original RGB'); axes[0].axis('off')
axes[1].imshow(gray_cv, cmap='gray');  axes[1].set_title('Grayscale');    axes[1].axis('off')
plt.tight_layout()
plt.show()

## 2. Comparing Grayscale Methods

Different methods give different results depending on how they weight the channels.

In [ ]:
R = img_rgb[:, :, 0].astype(np.float32)
G = img_rgb[:, :, 1].astype(np.float32)
B = img_rgb[:, :, 2].astype(np.float32)

# 1. Simple average
gray_avg = ((R + G + B) / 3).astype(np.uint8)

# 2. Luminance formula (BT.601)
gray_lum = (0.299 * R + 0.587 * G + 0.114 * B).astype(np.uint8)

# 3. Take only the red channel
gray_r = img_rgb[:, :, 0]

# 4. OpenCV result (for comparison)
gray_cv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for ax, im, t in zip(axes, 
    [gray_avg, gray_lum, gray_r, gray_cv],
    ['Average (R+G+B)/3', 'Luminance formula', 'Red channel only', 'OpenCV cvtColor']):
    ax.imshow(im, cmap='gray')
    ax.set_title(t)
    ax.axis('off')
plt.tight_layout()
plt.show()

## 3. Grayscale Color Maps

Matplotlib has different color maps for displaying single-channel images.  
If you don't specify `cmap='gray'`, it uses a default colored map.

In [ ]:
cmaps = ['gray', 'hot', 'jet', 'viridis', 'bone']
fig, axes = plt.subplots(1, len(cmaps), figsize=(18, 4))

for ax, cmap in zip(axes, cmaps):
    ax.imshow(gray_cv, cmap=cmap)
    ax.set_title(f'cmap={cmap}')
    ax.axis('off')

plt.suptitle('Same grayscale image — different color maps', fontsize=11)
plt.tight_layout()
plt.show()
# Note: 'jet' and 'hot' can make it look colored but it's still 1 channel

## 4. Converting Grayscale Back to 3-Channel

Some functions require a 3-channel input. We can "expand" a grayscale back to 3 identical channels.

In [ ]:
# Method 1: Use cv2.cvtColor
gray_3ch_cv = cv2.cvtColor(gray_cv, cv2.COLOR_GRAY2BGR)

# Method 2: Use numpy stack
gray_3ch_np = np.stack([gray_cv, gray_cv, gray_cv], axis=2)

print(f'Original gray: {gray_cv.shape}')
print(f'Expanded cv2:  {gray_3ch_cv.shape}')
print(f'Expanded np:   {gray_3ch_np.shape}')

# They should be identical
print(f'Same result?   {np.array_equal(gray_3ch_cv, gray_3ch_np)}')

## 5. Histogram of Grayscale Image

A grayscale histogram shows the distribution of pixel intensities (how many pixels are dark vs bright).

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].imshow(gray_cv, cmap='gray')
axes[0].set_title('Grayscale image')
axes[0].axis('off')

# Calculate and plot histogram
hist = cv2.calcHist([gray_cv], [0], None, [256], [0, 256])
axes[1].plot(hist, color='black')
axes[1].fill_between(range(256), hist[:, 0], alpha=0.5, color='steelblue')
axes[1].set_title('Intensity Histogram')
axes[1].set_xlabel('Pixel intensity (0=black, 255=white)')
axes[1].set_ylabel('Number of pixels')
axes[1].set_xlim([0, 256])
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f'Most common intensity: {hist.argmax()}')
print(f'Mean intensity:        {gray_cv.mean():.1f}')

## Summary

- `cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)` — standard way to convert to grayscale
- OpenCV uses the **luminance formula** (0.299R + 0.587G + 0.114B) — green has most weight
- Grayscale shape is `(H, W)` — no channel dimension
- Always use `cmap='gray'` in matplotlib for single-channel images
- Use `COLOR_GRAY2BGR` to expand grayscale back to 3-channel for functions that require it